In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.1. Data Imputation')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.1. Data Imputation/Output')
cur_date = "050623"

library(missForest)
library(imputeLCMD)
library(readxl)
library(tidyverse)
#library(tmvtnorm)

# reading in file
well_df = data.frame(read_excel("Input/Union_WellData_050623.xlsx", sheet = 2))

# converting some cols to numeric
numeric_df = data.frame(apply(well_df[,c(5,7,9,11)], 2, as.numeric))
recombined_df = cbind(well_df[,-c(5,7,9,11)], numeric_df)
well_df = recombined_df[,c(1:4,9,5,10,6,11,7,12,8)]

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“Expecting numeric in H1895 / R1895C8: got 'NA'”
Warning message:
“Expecting numeric in H1896 / R1896C8: got 'NA'”
Warning message:
“Expecting numeric in H1897 / R1897C8: got 'NA'”
Warning message in apply(well_df[, c(5, 7, 9, 11)], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(well_df[, c(5, 7, 9, 11)], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(well_df[, c(5, 7, 9, 11)], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(well_df[, c(5, 7, 9, 11)], 2, as.numeric):
“NAs introduced by coercion”


In [2]:
head(well_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Ar,NA,ND
2,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Mn,440,D
3,1006004,960,12-155,9/24/12,52,165,41,60,7.7,Cr,NA,ND
4,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Ar,NA,ND
5,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Mn,690,D
6,1024009,1803,15-177,12/17/15,40,445,42,2,7.3,Cr,NA,ND


Imputing missing data. Random forest will be used to input casing depth, well depth, static water depth, flow rate, and pH. Quantile regression imputation of left-censored data (QRILC) will be used to impute each metal concentration.

Typically background filters would be implemented for all data, but only the variable `Concentration` is missing a significant portion of data.

In [8]:
# splitting the dataset based on the metal
preimputed_df = well_df %>%
    group_by(Metal) %>%
    group_split()

arsenic_df = preimputed_df[[1]]
chromium_df = preimputed_df[[2]]
manganese_df = preimputed_df[[3]]

# RF Imputation

In [10]:
#imputing cols 5-9 using RF
random_forest_imputation = function(dataset){
    
    imputed_RF_object = missForest(as.matrix(dataset[,5:9]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:4], imputed_RF_df, dataset[,10:12])) 
    
    return(imputed_final_df)
}

# calling fn
imputed_arsenic_df = random_forest_imputation(arsenic_df)
imputed_chromium_df = random_forest_imputation(chromium_df)
imputed_manganese_df = random_forest_imputation(manganese_df)

head(imputed_arsenic_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60.0,7.7,Ar,NA,ND
2,1024009,1803,15-177,12/17/15,40,445,42,2.0,7.3,Ar,NA,ND
3,1054019,1647,14-212,2/2/15,45,160,40,40.0,7.4,Ar,NA,ND
4,1057017,970,12-169,10/22/12,42,440,57,1.5,8.0,Ar,35,D
5,1060006,533,10-239,1/3/11,48,120,42,25.0,7.1,Ar,NA,ND
6,1066006,1795,15-224,12/15/15,60,280,32,10.0,8.2,Ar,53,D


# QRILC Imputation